In [ ]:
import csv
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

lines = []
with open('./data/data/driving_log.csv') as csv_file:
    reader = csv.reader(csv_file)
    next(reader)
    for line in reader:
        lines.append(line)
        
X_train ,Y_train  = train_test_split(lines, test_size=0.2)
        
def generator(samples, batch_size=32):
    num_samples = len(samples)
   
    while 1: 
        shuffle(samples) #shuffling the total images
        for offset in range(0, num_samples, batch_size):
            
            lines = samples[offset:offset+batch_size]
            images = []
            measurements  = []

            for line in lines:
                for i in  range(3):
                    source_path = line[0]
                    file_name   = source_path.split('/')[-1]
                    current_path= './data/data/IMG/' + file_name
                    image       = cv2.imread(current_path)
                    images.append(image)
                    measurement  = float(line[3])
                    if(i==0):
                        measurements.append(measurement)
                    elif(i==1):
                        measurements.append(measurement + 0.2)
                    elif(i==2):
                        measurements.append(measurement - 0.2)
                    images.append(cv2.flip(image, 1))
                    if(i==0):
                        measurements.append(measurement * -1)
                    elif(i==1):
                        measurements.append((measurement + 0.2) * -1)
                    elif(i==2):
                        measurements.append((measurement - 0.2) * -1)
                        
                    
            X_train  = np.array(images)
            Y_train  = np.array(measurements)
            sklearn.utils.shuffle(X_train)
            sklearn.utils.shuffle(Y_train)
            yield X_train, Y_train
            

    
train_generator = generator(X_train, batch_size=32)
validation_generator = generator(Y_train, batch_size=32)

In [ ]:

from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers import Conv2D
from keras.layers import Lambda, Cropping2D

model   = Sequential()
model.add(Lambda(lambda x:(x / 255.0 ) - 0.5 , input_shape=(160,320,3)))
# trim image to only see section with road
model.add(Cropping2D(cropping=((70,25),(0,0))))           

#layer 1- Convolution, no of filters- 24, filter size= 5x5, stride= 2x2
model.add(Conv2D(24,(5,5),2))
model.add(Activation('relu'))

#layer 2- Convolution, no of filters- 36, filter size= 5x5, stride= 2x2
model.add(Conv2D(36,(5,5),2))
model.add(Activation('relu'))

#layer 3- Convolution, no of filters- 48, filter size= 5x5, stride= 2x2
model.add(Conv2D(48,(5,5),2))
model.add(Activation('relu'))

#layer 4- Convolution, no of filters- 64, filter size= 3x3, stride= 1x1
model.add(Conv2D(64,(3,3),1))
model.add(Activation('relu'))

#layer 5- Convolution, no of filters- 64, filter size= 3x3, stride= 1x1
model.add(Conv2D(64,(3,3),1))
model.add(Activation('relu'))

#flatten image from 2D to side by side
model.add(Flatten())

#layer 6- fully connected layer 1
model.add(Dense(100))
model.add(Activation('relu'))

#Adding a dropout layer to avoid overfitting. Here we are have given the dropout rate as 25% after first fully connected layer
model.add(Dropout(0.25))

#layer 7- fully connected layer 1
model.add(Dense(50))
model.add(Activation('relu'))


#layer 8- fully connected layer 1
model.add(Dense(10))
model.add(Activation('relu'))

#layer 9- fully connected layer 1
model.add(Dense(1)) #here the final layer will contain one value as this is a regression problem and not classification


# the output is the steering angle
# using mean squared error loss function is the right choice for this regression problem
# adam optimizer is used here
model.compile(loss='mse',optimizer='adam')


#fit generator is used here as the number of images are generated by the generator
# no of epochs : 7

model.fit_generator(train_generator, steps_per_epoch= len(X_train), validation_data=validation_generator,   validation_steps=len(Y_train), epochs=7, verbose=2)

#saving model
model.save('model.h5')

print('Done! Model Saved!')

# keras method to print the model summary
model.summary()

In [ ]:
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()